The Black-Sholes model is used to determine the price of financial derivatives, such as the price of an option for a given underlying stock.

From Investopedia:
"_Black-Scholes posits that instruments, such as stock shares or futures contracts, will have a lognormal distribution of prices following a random walk with constant drift and volatility. Using this assumption and factoring in other important variables, the equation derives the price of a European-style call option._

_The Black-Scholes equation requires five variables. These inputs are volatility, the price of the underlying asset, the strike price of the option, the time until expiration of the option, and the risk-free interest rate. With these variables, it is theoretically possible for options sellers to set rational prices for the options that they are selling._

_Furthermore, the model predicts that the price of heavily traded assets follows a geometric Brownian motion with constant drift and volatility. When applied to a stock option, the model incorporates the constant price variation of the stock, the time value of money, the option's strike price, and the time to the option's expiry._"

### Black-Scholes Assumptions

- No dividends are paid out during the life of the option.
- Markets are random (i.e., market movements cannot be predicted).
- There are no transaction costs in buying the option.
- The risk-free rate and volatility of the underlying asset are known and constant.
- The returns of the underlying asset are normally distributed.
- The option is European and can only be exercised at expiration.

While the Black-Sholes model is itself a stochastic differential equiation, the solution to the equation can be easily implemented

## Basic implementation

In [136]:
from scipy import stats
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
S0 = 100  # Initial price
E = 100  # Strike price
rf = 0.05  # Risk-free rate
sigma = 0.2  # Volatility
T = 1  # Time of maturity

In [137]:
def call_option_price(S, E, T, rf, sigma):
    """Calculate the price of a call option at time t=0 for a given underlying stock.

    Args:
        S: Stock price
        E: Exercise or Strike price
        T: Time horizon
        rf: Risk-free rate
        sigma: Standard deviation for stochastic process
    """
    d1 = (np.log(S / E) + (rf + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    St = S * stats.norm.cdf(d1) - E * np.exp(-rf * T) * stats.norm.cdf(d2)
    return St

In [138]:
def put_option_price(S, E, T, rf, sigma):
    """Calculate the price of a put option at time t=0 for a given underlying stock.

    Args:
        S: Stock price
        E: Exercise or Strike price
        T: Time horizon
        rf: Risk-free rate
        sigma: Standard deviation for stochastic process
    """
    d1 = (np.log(S / E) + (rf + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    St = -S * stats.norm.cdf(-d1) + E * np.exp(-rf * T) * stats.norm.cdf(-d2)
    return St

In [139]:
print(call_option_price(S0, E, T, rf, sigma))
print(put_option_price(S0, E, T, rf, sigma))

10.450583572185565
5.573526022256971


## Using Monte-Carlo

In [140]:
def geometric_random_walk(S0, T: int = 2, N: int = 1000, mu=0.1, sigma=0.5):
    """Simulate a geometric random walk.

    Args:
        S0: Initial position (i.e. initial stock price). Must be non-zero.
        T: Time horizon
        N: Number of samples within horizon
        mu: drift term
        sigma: stochastic term
    """
    dt = T / N
    t = np.linspace(0, T, N)
    W = np.random.standard_normal(N)
    # N(0,dt) ~ N(0,1) * sqrt(dt)
    W = np.cumsum(W) * np.sqrt(dt)
    X = (mu - 0.5 * sigma**2) * t + sigma * W
    S = S0 * np.exp(X)
    return S, t

Simulate a number of random walks to determine price at t=T

In [141]:
Ss = []
iterations = 10000

for _ in range(iterations):
    # Assume drift is risk-free rate
    S, t = geometric_random_walk(S0=S0, T=T, mu=rf, sigma=sigma)
    Ss.append(S[-1])  # Stock price at maturity

Calculate the price of the option at time t=T, and use a discount factor to calculate present value of the option.

In [142]:
# Call option
Ss = np.array(Ss)
# max(0,S-E)
average_future_price = np.mean(np.amax(list(zip(np.zeros(Ss.shape), Ss - E)), axis=1))
discount_factor = np.exp(-rf * T)
price = discount_factor * average_future_price
price

10.501937287451021

In [143]:
# Put option
Ss = np.array(Ss)
# max(0,E-S)
average_future_price = np.mean(np.amax(list(zip(np.zeros(Ss.shape), E - Ss)), axis=1))
discount_factor = np.exp(-rf * T)
price = discount_factor * average_future_price
price

5.508259158336803